In [1]:
import warnings
warnings.filterwarnings("ignore")
from shapely import wkt
import glob


%load_ext autoreload
%autoreload 2
%env PROJ_LIB=/home/wwymak/anaconda3/envs/solaris/share/proj
%matplotlib inline

import skimage

from tqdm import tqdm
import numpy as np
import cv2
from functools import partial

from fastai.imports import *
from fastai.vision import *
from fastai.metrics import dice
from fastai.callbacks import *

from joblib import Parallel, delayed
import torch.nn.functional as F
import torch
import functools, traceback
def gpu_mem_restore(func):
    """Reclaim GPU RAM if CUDA out of memory happened, or execution was interrupted"""
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except:
            type, val, tb = sys.exc_info()
            traceback.clear_frames(tb)
            raise type(val).with_traceback(tb) from None
    return wrapper

os.environ['FASTAI_TB_CLEAR_FRAMES']="1"

from catalyst.contrib.criterion import FocalLossMultiClass, DiceLoss, BCEDiceLoss, LovaszLossBinary,LovaszLossMultiClass

from importlib import reload

env: PROJ_LIB=/home/wwymak/anaconda3/envs/solaris/share/proj


In [2]:
data_dir = Path("/media/wwymak/Storage/xView2")
mask_dir = data_dir /"mask_full_size"
models_path = data_dir / "models"
train_images_crops = data_dir/"train_crops"
mask_dir_edt = data_dir /"mask_full_size_edt"
train_mask_crops = data_dir/"mask_crops_single_channel"
label_dir = data_dir/"train"/"labels"
mask_crops_edt = data_dir/"mask_crops_edt"

In [3]:
data_dir = Path("/media/wwymak/Storage/xView2")
models_path = data_dir / "models"
classifcation_crop_dir = data_dir/"classification_crops"
classification_labels = pd.read_csv(data_dir/"train.csv")

damage_crops_test_folder = data_dir / "classification_crops_test"
classification_labels.head()

,uuid,img_id,label,crop_filename
0,c18d34cd-d48d-43ba-9d9c-011b6f9d5eb8,palu-tsunami_00000024_post_disaster.png,0,c18d34cd-d48d-43ba-9d9c-011b6f9d5eb8.png
1,4db83f7b-7c77-4cb4-8ea7-ad5a338a92b1,palu-tsunami_00000024_post_disaster.png,0,4db83f7b-7c77-4cb4-8ea7-ad5a338a92b1.png
2,fc34426b-db11-403d-bdcf-1527b7d2508a,palu-tsunami_00000024_post_disaster.png,0,fc34426b-db11-403d-bdcf-1527b7d2508a.png
3,052f9f14-e32e-484a-811d-cd0fac134493,palu-tsunami_00000024_post_disaster.png,0,052f9f14-e32e-484a-811d-cd0fac134493.png
4,577576f3-3bdc-41ec-a571-6f6cf843cf3b,palu-tsunami_00000024_post_disaster.png,0,577576f3-3bdc-41ec-a571-6f6cf843cf3b.png


In [4]:
classification_labels.label.value_counts()

0    120419
1     14980
2     14161
3     13227
Name: label, dtype: int64

In [6]:
def is_valid_func(fname, valid_ids):
    
    img_id = Path(fname).name.replace('rgb_', '').split('.')[0]

    return img_id in valid_ids

def get_mask_fn(fname):
    return train_mask_crops / fname.name.replace("rgb", "mask")

In [5]:
unique_img_ids_undamaged= list(set([x.name.replace('mask_', '').split('.')[0] for x in train_mask_crops.ls() if 'pre' in x.name ]))
unique_img_ids_damaged= list(set([x.name.replace('mask_', '').split('.')[0] for x in train_mask_crops.ls() if 'post' in x.name ]))

print(len(unique_img_ids_damaged), len(unique_img_ids_undamaged))

np.random.seed(42)
np.random.shuffle(unique_img_ids_undamaged)
train_ids_undamaged = unique_img_ids_undamaged[:int(0.8 * len(unique_img_ids_undamaged))]
valid_ids_undamaged = unique_img_ids_undamaged[int(0.8 * len(unique_img_ids_undamaged)):]

train_ids_damaged = [x.replace('pre', 'post') for x in train_ids_undamaged]
valid_ids_damaged =[x.replace('pre', 'post') for x in valid_ids_undamaged]

print(len(train_ids_damaged), len(valid_ids_damaged))

2283 2283
1826 457


In [ ]:
datasets= []
classification_labels_0 = classification_labels[classification_labels.label==0].sample(frac=1.)
label_0_per_bag = 20000
for i in range(0, len(classification_labels_0) , label_0_per_bag):
    datasets.append(pd.concat([
        classification_labels[(classification_labels.label!= 0)][i: i+label_0_per_bag],
        classification_labels[(classification_labels.label==0)]
    ]).reset_index(drop=True))
